In [1]:
#append system dir folder
import sys
from dependencies import *
keyword_list = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon"]


In [2]:
async def messages(chat_id, session):
    app = Client(session)
    messages_info = []
    try:
        async with app:
            #how to get odd number of messages
            async for message in app.search_messages(chat_id, limit=50000):
                if message.text:
                    messages_info.append(message.text.lower())
            print(f"messages: {chat_id} done")
            return messages_info
    except Exception as e:
        print(f"messages: Error in {chat_id}: {e}")
        return None
                

In [3]:
async def relevance_score(chat_id, keyword_list, threshold=0.1):
    tasks = [messages(chat_id, session) for session in sessions]
    messages_info_list = await asyncio.gather(*tasks)
    #if every sublist is none, return None
    if all([item is None for item in messages_info_list]):
        return None
    #flatten the list, exclude None lists
    messages_info = [item for sublist in messages_info_list if sublist is not None for item in sublist]
    messages_info = list(set(messages_info))
    
    message_count = len(messages_info)
    found_keywords = set()
    found = 0
    
    for message in messages_info:
        found_keywords.update([keyword for keyword in keyword_list if message.lower().count(keyword) > 0])
        found += sum([message.lower().count(keyword) for keyword in keyword_list])
        
    keywords_count = len(found_keywords)
    message_count = len(messages_info)
    
    relevance_score = (found / message_count) * (1 + 0.5 * (keywords_count / len(keyword_list)))
    raw_score = keywords_count / len(keyword_list)

    if raw_score <= threshold:
        print("Not that much Relevant")
        relevance_score -= 1
    else:
        print("Highly Relevant")
        
    result ={"ch_id": chat_id,"relevance_score": relevance_score, "raw_score": raw_score,"keywords_count": keywords_count, "found_keywords": list(found_keywords)}
    #ch_id,relevance_score,raw_score,keywords_count,found_keywords
    return result

In [ ]:
relevant_chats = pd.read_csv('relevant.csv')
chats = pd.read_csv('chat_info.csv')
chat_ids = chats['ch_id'].tolist()
sessions = [ "my_account", "p_account"]

for chat_id in chat_ids[1317:]:
    result = await relevance_score(chat_id, keyword_list)
    if result == None:
        continue
    
    result_df = pd.DataFrame([result])
    result_df.to_csv('relevant.csv', mode='a', header=False, index=False)

In [16]:
#drop duplicates
relevant_chats = pd.read_csv('relevant.csv')
relevant_chats = relevant_chats.drop_duplicates()
relevant_chats.to_csv('relevant.csv', index=False)